In [1]:
import torch
from torch.utils.data import Dataset
import numpy as np
from pathlib import Path
from tqdm.notebook import tqdm_notebook

In [2]:
class MRIDataset(Dataset):
    def __init__(self, path_to_data, split="train"):
        if split == "train":
            split_indices = [*range(1, 101)]
        elif split == "test":
            split_indices = [*range(101, 111)]

        print(f"Start reading {split}ing dataset...")

        # read the first set of volumes
        clean_mri_set = np.load(path_to_data / "data/1.npy")
        noisy_mri_set = np.load(path_to_data / "noisy/1.npy")

        for i in tqdm_notebook(split_indices):
            # load the current volumes
            clean_mri_temp = np.load(path_to_data / f"data/{i}.npy")
            noisy_mri_temp = np.load(path_to_data / f"noisy/{i}.npy")

            # append to the existing stack
            clean_mri_set = np.append(clean_mri_set, clean_mri_temp, axis=0)
            noised_mri_set = np.append(noisy_mri_set, noisy_mri_temp, axis=0)

        self.clean_mri_set = clean_mri_set
        self.noisy_mri_set = noisy_mri_set
        self.total = self.clean_mri_set.shape[0]
        self.current_patch = 1

        print(self.clean_mri_set.shape)
        print(f"End reading {split}ing dataset...")

    def __len__(self):
        return self.total

    def __getitem__(self, index):
        clean_img = torch.from_numpy(self.clean_mri_set[index]).float()
        noisy_img = torch.from_numpy(self.noisy_mri_set[index]).float()
        return {"clean_img": clean_img, "noisy_img": noisy_img}

In [3]:
path_to_data = Path("../../patches/")

train_dataset = MRIDataset(path_to_data=path_to_data, split="train")

Start reading training dataset...


  0%|          | 0/100 [00:00<?, ?it/s]

(79184, 1, 1, 6, 32, 32)
End reading training dataset...


In [4]:
test_dataset = MRIDataset(path_to_data=path_to_data, split="test")

Start reading testing dataset...


  0%|          | 0/10 [00:00<?, ?it/s]

(8624, 1, 1, 6, 32, 32)
End reading testing dataset...
